In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

# suppress warnings
import warnings
warnings.filterwarnings('ignore')


In [8]:
# read the data
df = pd.read_csv('https://raw.githubusercontent.com/nikjohn7/Disaster-Tweets-Kaggle/main/data/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
# how many rows and columns are in the data set?
df.shape

(7613, 5)

In [11]:
# how many tweets about disasters
df[df['target'] == 1].shape

(3271, 5)

In [12]:
# how many tweets not about disasters
df[df['target'] == 0].shape

(4342, 5)

In [14]:
import nltk
stopwords = set(nltk.corpus.stopwords.words('english'))

include_stopwords = {'disaster', 'natural'}
stopwords |= include_stopwords

In [18]:
# build a text processing and classifier pipeline
# to predict the country (USA or Canada) of a speech

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2)

# Create a pipeline that first transforms the text data into TF-IDF vectors, then applies SVM
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', svm.SVC()),
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred, target_names=[]))


In [16]:
# This script creates a new column 'sentiment' in the dataframe, 
# which contains the sentiment score of the text. 
# The sentiment score is a float within the range [-1.0, 1.0], 
# where -1.0 denotes a very negative sentiment, 
# 1.0 denotes a very positive sentiment, 
# and values around 0 denote a neutral sentiment.

from textblob import TextBlob

# Define a function to apply sentiment analysis to a text
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # returns a value between -1 and 1

# Create a new column 'sentiment' in the DataFrame
df2['sentiment'] = df2['text'].apply(get_sentiment)

# Display the DataFrame
# df2

In [17]:
# find average sentiment for each country in df2
df2.groupby('country')['sentiment'].mean()

country
CAN    0.112540
USA    0.110635
Name: sentiment, dtype: float64

In [18]:
# find average sentiment for each speaker in df2
# order the results from most positive to most negative

df2.groupby('speaker')['sentiment'].mean().sort_values(ascending=False).head(5)

speaker
Condoleezza Rice    0.164408
Lawrence Cannon     0.152515
Leonard Edwards     0.145419
Mr. MACGUIGAN       0.144525
Jean Chrétien       0.142099
Name: sentiment, dtype: float64

In [19]:
df2.groupby('year')['sentiment'].mean().sort_values(ascending=False)

year
2008    0.143186
1989    0.140575
1991    0.139629
2003    0.137859
1981    0.135435
1971    0.129006
1994    0.128335
1993    0.126281
2006    0.125876
1995    0.125568
1973    0.125375
1997    0.124643
2015    0.123486
2010    0.122846
2005    0.121935
1987    0.121119
2014    0.119271
1998    0.117136
2009    0.116599
2007    0.116335
1970    0.114943
1983    0.114040
1978    0.113078
1976    0.111378
1977    0.111315
1984    0.110240
1996    0.109673
1988    0.109235
1992    0.107583
2004    0.106383
1990    0.104914
1985    0.103662
1972    0.100319
2001    0.099462
2011    0.099054
1974    0.098554
1980    0.097081
2000    0.095660
1979    0.095185
1982    0.093348
1999    0.093205
1975    0.090080
2002    0.084102
1986    0.084058
2012    0.074286
2013    0.071692
Name: sentiment, dtype: float64